In [2]:
from pdf2image import convert_from_path
import pytesseract
import re

In [3]:
# Path to your PDF
pdf_path = "success_prediction/shab_history_extraction/sha-002_2015_133__12686_d.pdf"

# Convert PDF page to image
pages = convert_from_path(pdf_path, dpi=300)  # higher dpi for better OCR

In [27]:
test = pytesseract.image_to_string(pages[39], lang='deu')

In [28]:
test

'Die Gläubiger des Schuldners und alle Personen, die Ansprüche auf die\nin seinem Besitz befindlichen Vermögensstücke haben, werden aufgefor-\ndert, ihre Forderungen oder Ansprüche samt Beweismitteln (Schuld-\nscheine, Buchauszüge usw.) innert der Eingabefrist dem betreffenden\nKonkursamt einzureichen. Mit der Eröffnung des Konkurses hört gegen-\nüber dem Schuldner der Zinsenlauf auf. Für pfandgesicherte Forderun-\n‚gen läuft der Zins bis zur Verwertung weiter, soweit der Pfanderlös den\nBetrag der Forderung und des bis zur Konkurseröffnung aufgelaufenen\nZinses übersteigt (Art. 209 SchKG). Die Grundpfandgläubiger haben ihre\nForderungen in Kapital, Zinsen und Kosten zerlegt anzumelden und\ngleichzeitig auch anzugeben, ob die Kapitalforderung schon fällig oder\ngekündigt sei, allfällig für welchen Betrag und auf welchen Termin. Die In-\nhaber von Dienstbarkeiten, welche unter dem früheren kantonalen Recht\n‚ohne Eintragung in die öffentlichen Bücher entstanden und noch nicht im\nGrundb

In [4]:
ocr_texts = []
for i in range(1, len(pages)):
    text = pytesseract.image_to_string(pages[i], lang='deu').split('\n\n', 1)[-1]
    ocr_texts.append(re.search(r"Registro di commercio\s+(.*?)(\n\n\[U \}\nRedaktion/Verlag)", text, re.DOTALL).group(1) if i == 1 else text)

full_text = ''.join(ocr_texts)

In [21]:
print([m.span() for m in re.finditer("Westcom AG, in Zug", full_text)])

[(860821, 860839), (867622, 867640)]


In [ ]:
separators = {
    "Neueintragungen\nNouvelles inscriptions\nNuove iscrizioni": 'new_inscriptions',
    "Mutationen\nMutations\nMutazioni": 'mutations',
    "Löschungen\nRadiations\nCancellazioni": 'deletions',
}

slices = [m.span() for m in re.finditer('|'.join(separators.keys()), full_text)]

text_slices = {'new_inscriptions': [], 'mutations': [], 'deletions': []}
for i in range(len(slices)-1):

    slice_i = slices[i]
    slice_j = slices[i+1]

    keyword = full_text[slice_i[0]:slice_i[1]]
    
    if i == len(slices) - 1:
        text_slices[separators.get(keyword)].append(full_text[slice_i[1]:])
    else:
        text_slices[separators.get(keyword)].append(full_text[slice_i[1]:slice_j[0]])

In [ ]:
new_inscriptions = []
for text in text_slices['new_inscriptions']:
    entries = re.split(r'Tagesregister-Nr', text)
    cleaned_entries = [entry.split('= ', 1) for entry in entries]
    new_inscriptions.extend([entry[1] for entry in cleaned_entries if len(entry) == 2])

mutations = []
for text in text_slices['mutations']:
    entries = re.split(r'Tagesregister-Nr', text)  
    cleaned_entries = [entry.split('= ', 1) for entry in entries]
    mutations.extend([entry[1] for entry in cleaned_entries if len(entry) == 2])

deletions = []
for text in text_slices['deletions']:
    entries = re.split(r'Tagesregister-Nr', text)  
    cleaned_entries = [entry.split('= ', 1) for entry in entries]
    deletions.extend([entry[1] for entry in cleaned_entries if len(entry) == 2])

In [ ]:
new_inscriptions